# PYTTI: Python Text-to-Image 

AI-Assisted Artistic Image Generation and Manipulation

# 1. Setup

Setup instructions can be found here: https://pytti-tools.github.io/pytti-book/intro.html

# 2: Define Parameters (i.e. the inputs to this tool)

Settings documentation can be found here:
* [Scene Syntax](https://pytti-tools.github.io/pytti-book/SceneDSL.html)
* [Everything Else](https://pytti-tools.github.io/pytti-book/Settings.html)

Please modify the values in the `config/` folder's yaml files, the files will be read when you execute the main "workhorse" cell.

Feel free to modify the values in `config/default.yaml`. If you have discovered alternative default settings that you prefer, feel free to persist them in that file. You can then specify your runs by authoring files that define just the parameters you want to change in the `config/conf/` folder. An example `demo.yaml` is provided. 

In the following cell, we specify the name of the config file containing the overrides to the default settings. To use override settings other than those contained in `config/conf/demo.yaml`, specify the correct filename below. 

Descriptions of parameter options and features can be found at the bottom of this notebook.

In [ ]:
# If multiple filenames are present, they will be looped
CONFIG_OVERRIDES = [] # ["demo.yaml"] # File(s) located in config/conf

# You probably don't need to touch these
CONFIG_BASE_PATH = "config"
CONFIG_DEFAULTS = "default.yaml"

# If randomize_seed is set to False, two runs with the same parameters will produce the same outputs
#RANDOMIZE_SEED = False # feature not yet supported.

##################

## Don't change values in this cell below this line

# overrides to let you paste raw json
null=None
true=True
false=False

# added to allow users not pasting in settings to skip that cell
pytti_panna_output = {}

If you would like to use parameters generated by pyttipanna or an older version of the notebook, incorporate them below

In [ ]:
# Replace '{}' with your pytti panna thing
pytti_panna_output = {}

# 3. (Optional) Monitor Outputs with Tensorboard

This notebook supports tensorboard outputs. These are the same values that will be printed while the notebook runs, but they may be easier to navigate in tensorboard if you want to start a server. Uncomment the cell below to initialize tensorboard in the notebook.

In [ ]:
#from pytti.workhorse import _main, TB_LOGDIR 

#%load_ext tensorboard
#%tensorboard --logdir $TB_LOGDIR 

# 4. RUN IT!

Just execute the cell below

In [ ]:
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from loguru import logger
from omegaconf import OmegaConf
from pytti.workhorse import _main as render_frames

# Necessary for VS Code to respect displayed outputs
%matplotlib inline

try:
    assert isinstance(CONFIG_OVERRIDES, list)
except AssertionError:
    if isinstance(CONFIG_OVERRIDES, str):
        logger.warning(
            "The CONFIG_OVERRIDES variable should be a list of filenames."
            "I noticed you set this variable to a string. I'll wrap that in "
            "a list for you this time, but after this cell completes execution, "
            "please repair how you set the variable above. Instead of"
            f'\n\n\tCONFIG_OVERRIDES="{CONFIG_OVERRIDES}\n\n'
            "it should be"
            f'\n\n\tCONFIG_OVERRIDES=["{CONFIG_OVERRIDES}"]\n\n'
        )
        CONFIG_OVERRIDES = [CONFIG_OVERRIDES]

# https://github.com/facebookresearch/hydra/blob/main/examples/jupyter_notebooks/compose_configs_in_notebook.ipynb
# https://omegaconf.readthedocs.io/
# https://hydra.cc/docs/intro/
with initialize(config_path=CONFIG_BASE_PATH):
    if pytti_panna_output:
        cfg = OmegaConf.create(pytti_panna_output)
        render_frames(cfg)
    for c_o in CONFIG_OVERRIDES:
        cfg = compose(config_name=CONFIG_DEFAULTS, 
                      overrides=[f"conf={c_o}"])
        render_frames(cfg)

# 4. Render video

The first run executed in `file_namespace` (probably set to `default`) is number $0$, the second is number $1$, etc.

In [ ]:
# quick fix
import re
params = cfg

from os.path import exists as path_exists
#if path_exists('/content/drive/MyDrive/pytti_test'):
#  %cd /content/drive/MyDrive/pytti_test
#  drive_mounted = True
#else:
drive_mounted = False
#try:
#  from pytti.Notebook import change_tqdm_color
#except ModuleNotFoundError:
#  if drive_mounted:
#    #THIS IS NOT AN ERROR. This is the code that would
#    #make an error if something were wrong.
#    raise RuntimeError('ERROR: please run setup (step 1.3).')
#  else:
#    #THIS IS NOT AN ERROR. This is the code that would
#    #make an error if something were wrong.
#    raise RuntimeError('WARNING: drive is not mounted.\nERROR: please run setup (step 1.3).')
#change_tqdm_color()
  
from tqdm.notebook import tqdm
import numpy as np
from os.path import exists as path_exists
from subprocess import Popen, PIPE
from PIL import Image, ImageFile
from os.path import splitext as split_file
import glob
from pytti.Notebook import get_last_file

ImageFile.LOAD_TRUNCATED_IMAGES = True

try:
  params
except NameError:
  raise RuntimeError("ERROR: no parameters. Please run parameters (step 2.1).")

if not path_exists(f"images_out/{params.file_namespace}"):
  if path_exists(f"/content/drive/MyDrive"):
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError(f"ERROR: file_namespace: {params.file_namespace} does not exist.")
  else:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError(f"WARNING: Drive is not mounted.\nERROR: file_namespace: {params.file_namespace} does not exist.")

#@markdown The first run executed in `file_namespace` is number $0$, the second is number $1$, etc.

latest = -1
run_number = latest#@param{type:"raw"}
if run_number == -1:
  _, i = get_last_file(f'images_out/{params.file_namespace}', 
                       f'^(?P<pre>{re.escape(params.file_namespace)}\\(?)(?P<index>\\d*)(?P<post>\\)?_1\\.png)$')
  run_number = i
base_name = params.file_namespace if run_number == 0 else (params.file_namespace+f"({run_number})")
tqdm.write(f'Generating video from {params.file_namespace}/{base_name}_*.png')

all_frames = glob.glob(f'images_out/{params.file_namespace}/{base_name}_*.png')
all_frames.sort(key = lambda s: int(split_file(s)[0].split('_')[-1]))
print(f'found {len(all_frames)} frames matching images_out/{params.file_namespace}/{base_name}_*.png')

start_frame = 0#@param{type:"number"}
all_frames = all_frames[start_frame:]

fps =  params.frames_per_second#@param{type:"raw"}

total_frames = len(all_frames)

if total_frames == 0:
  #THIS IS NOT AN ERROR. This is the code that would
  #make an error if something were wrong.
  raise RuntimeError(f"ERROR: no frames to render in images_out/{params.file_namespace}")

frames = []

for filename in tqdm(all_frames):
  frames.append(Image.open(filename))

p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '1', '-preset', 'veryslow', f"videos/{base_name}.mp4"], stdin=PIPE)
for im in tqdm(frames):
  im.save(p.stdin, 'PNG')
p.stdin.close()

print("Encoding video...")
p.wait()
print("Video complete.")

In [ ]:
#@title 3.1 Render video (concatenate all runs)
#from os.path import exists as path_exists
#if path_exists('/content/drive/MyDrive/pytti_test'):
#  %cd /content/drive/MyDrive/pytti_test
#  drive_mounted = True
#else:
#  drive_mounted = False
#try:
#  from pytti.Notebook import change_tqdm_color
#except ModuleNotFoundError:
#  if drive_mounted:
#    #THIS IS NOT AN ERROR. This is the code that would
#    #make an error if something were wrong.
#    raise RuntimeError('ERROR: please run setup (step 1.3).')
#  else:
#    #THIS IS NOT AN ERROR. This is the code that would
#    #make an error if something were wrong.
#    raise RuntimeError('WARNING: drive is not mounted.\nERROR: please run setup (step 1.3).')
#change_tqdm_color()
  
from tqdm.notebook import tqdm
import numpy as np
from os.path import exists as path_exists
from subprocess import Popen, PIPE
from PIL import Image, ImageFile
from os.path import splitext as split_file
import glob
from pytti.Notebook import get_last_file

ImageFile.LOAD_TRUNCATED_IMAGES = True

try:
  params
except NameError:
  raise RuntimeError("ERROR: no parameters. Please run parameters (step 2.1).")

#if not path_exists(f"images_out/{params.file_namespace}"):
#  if path_exists(f"/content/drive/MyDrive"):
#    raise RuntimeError(f"ERROR: file_namespace: {params.file_namespace} does not exist.")
#  else:
#    raise RuntimeError(f"WARNING: Drive is not mounted.\nERROR: file_namespace: {params.file_namespace} does not exist.")

#@markdown The first run executed in `file_namespace` is number $0$, the second is number $1$, etc.

latest = -1
run_number = latest
if run_number == -1:
  _, i = get_last_file(f'images_out/{params.file_namespace}', 
                       f'^(?P<pre>{re.escape(params.file_namespace)}\\(?)(?P<index>\\d*)(?P<post>\\)?_1\\.png)$')
  run_number = i

all_frames = []
for i in range(run_number+1):
  base_name = params.file_namespace if i == 0 else (params.file_namespace+f"({i})")
  frames = glob.glob(f'images_out/{params.file_namespace}/{base_name}_*.png')
  frames.sort(key = lambda s: int(split_file(s)[0].split('_')[-1]))
  all_frames.extend(frames)

start_frame = 0#@param{type:"number"}
all_frames = all_frames[start_frame:]

fps =  params.frames_per_second#@param{type:"raw"}

total_frames = len(all_frames)

if total_frames == 0:
  #THIS IS NOT AN ERROR. This is the code that would
  #make an error if something were wrong.
  raise RuntimeError(f"ERROR: no frames to render in images_out/{params.file_namespace}")

frames = []

for filename in tqdm(all_frames):
  frames.append(Image.open(filename))

p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '1', '-preset', 'veryslow', f"videos/{base_name}.mp4"], stdin=PIPE)
for im in tqdm(frames):
  im.save(p.stdin, 'PNG')
p.stdin.close()

print("Encoding video...")
p.wait()
print("Video complete.")

In [ ]:
# Optionally display your video in the notebook

from IPython.display import Video

output_video_filename = "default.mp4" # you'll probably need to change this after the first run

Video(output_video_filename)

# Batch Setings

Get fancy with a for loop or try using hydra's `--multi-run` feature. Improved documentation for lcoal batch jobs forthcoming.

# Debugging tools

In [ ]:
# Run this cell if you think your environment is broken

%conda info --envs

import warnings

try:
    import torch
    torch.cuda.is_available()
except ImportError as e:
    warnings.warn("Please install pytorch with CUDA (GPU): https://pytorch.org/get-started/locally/")
    warnings.warn("This is probably what you want: ")

try:
    import PIL
except ImportError as e:
    warnings.warn("Please install the Python Image Library: conda install -c conda-forge pillow")

try:
    import cv2
except ImportError as e:
    warnings.warn("Please install OpenCV: conda install -c conda-forge opencv")

try:
    import tensorflow as tf
except ImportError as e:
    warnings.warn("Please install Tensorflow with CUDA (GPU): conda install tensorflow-gpu")

try:
    import transformers
except:
    warnings.warn("Please install huggingface/transformers: conda install -c huggingface transformers")


#conda install -c conda-forge imageio
#conda install scikit-learn
#pip install gdown einops
#conda install pytorch-lightning -c conda-forge
#conda install -c conda-forge kornia
#conda install pandas 
#conda install seaborn
#pip install PyGLM
#pip install ftfy regex tqdm hydra-core adjustText exrex bunch matplotlib-label-lines
#git clone https://github.com/pytti-tools/pytti-core
#pip install ./AdaBins
#pip install ./GMA
#pip install ./CLIP
#pip install ./pytti-core